In [1]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [2]:
def search_scrape_ebay(query):
    chrome_options = Options()
    chrome_options.add_experimental_option("detach", True)
    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://ebay.com")
    driver.maximize_window()

    # search bar
    search_bar = driver.find_element(By.ID, "gh-ac")
    search_bar.send_keys(query)
    search_bar.send_keys(Keys.ENTER)

    # sold button
    sold_checkbox = driver.find_element(By.CSS_SELECTOR, "input[aria-label='Sold Items']")
    sold_checkbox.click()

    # find image + titles + prices + URL
    images = driver.find_elements(By.XPATH, "//div[@class='s-item__image-wrapper image-treatment']/img")
    titles = driver.find_elements(By.CSS_SELECTOR, "span[role='heading']")
    prices = driver.find_elements(By.CSS_SELECTOR, "span[class='s-item__price']")
    urls = driver.find_elements(By.CSS_SELECTOR, "a[class='s-item__link']")

    # calculate price markers
    num_listings = len(titles)
    max_price = 0.0
    sum_prices = 0.0
    for i in range(1, num_listings):
        pattern = r'(\d+\.\d{2})'
        match = re.search(pattern, prices[i].text)
        price = float(match.group())
        max_price = max(max_price, price)
        sum_prices += price;
    avg_price = sum_prices / num_listings

    # format to two decimal places
    max_price = float("{:.2f}".format(max_price))
    avg_price = float("{:.2f}".format(avg_price))
    
    sold_listings_url = driver.current_url
    
    driver.quit()
    
    return (sold_listings_url, max_price, avg_price)
